In [1]:
from AutoML import AutoML, DataNet, AutoMLDataset
#DataNet().restart_datanodes()
classes_semantics = [113988, 113989, 113990]
dataset = AutoMLDataset(name="person_classification")#, semantics=classes_semantics, crops=True) #, override=True)

/home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/reminiz/ReminizML2/python_env/lib/pyt

In [2]:
import ray
ray.init(address="auto", ignore_reinit_error=True)

2021-01-12 17:47:48,551	INFO worker.py:634 -- Connecting to existing Ray cluster at address: 192.168.8.51:6379
2021-01-12 17:47:48,552	ERROR worker.py:643 -- Calling ray.init() again after it has already been called.


In [3]:
add_images_stage = 5000

In [4]:
# get the indexes to emulate the labeled set

import numpy as np
import random

list_classes = list(dataset.dataset_header['train_images'].keys())
nb_classes = len(list_classes)        
classes={}
files = []
labels = []
nb_img_per_class = np.zeros(nb_classes, dtype = np.int32)
classes_cardinality = {}

for class_name in list_classes:
    class_index = list_classes.index(class_name)
    classes_cardinality[class_name] = len(dataset.dataset_header['train_images'][class_name])
    nb_img_per_class[class_index] = classes_cardinality[class_name]
    start = len(files)
    id_files = dataset.dataset_header["train_images"][class_name]
    files+=id_files
    labels+=[class_index] * len(id_files)
    end = len(files)
    classes[class_index]=range(start, end)

    
all_files = np.array(files)
all_labels = np.array(labels)
indices = list(range(len(files)))
random.seed(101)
random.shuffle(indices)
labeled_set = indices[:add_images_stage ]
unlabeled_set = indices[add_images_stage :]


In [5]:
# load defaulft config
import yaml

config_path = '/mnt/Ressources/Andres/Temp_active/configs/Datanet_random_2.yml'

with open(config_path) as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    config = yaml.load(file, Loader=yaml.FullLoader)

print(config)

{'DATASET': {'Data_augementation': True, 'original_size': 256, 'pad': False, 'random_crop': True, 'random_crop_pad': 30, 'random_flip': True, 'random_greyscale': False, 'random_hue': False, 'rot90': False, 'random_brightness': False, 'random_saturation': False}, 'NETWORK': {'Backbone': 'ResNet50', 'INPUT_SIZE': 256, 'MARGIN': 1.0, 'embedding_size': 128}, 'PROJECT': {'dataset_name': 'person_classification', 'group': 'Random_v2', 'group_dir': '/mnt/Ressources/Andres/runs/Random_v2', 'project': 'Active_Learning_Datanet', 'project_dir': '/mnt/Ressources/Andres/runs'}, 'TEST': {'batch_size': 16}, 'TRAIN': {'EPOCH_SLIT': 80, 'EPOCH_WARMUP': 2, 'EPOCH_WHOLE': 120, 'MILESTONES': [140, 180], 'batch_size': 32, 'lr': 0.0025, 'test_each': 5, 'transfer_weight_path': '/mnt/Ressources/Andres/runs/imagenet/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5', 'weight_lossnet_loss': 0.0, 'weight_decay': 2e-05, 'apply_weight_decay': False}}


In [6]:
import time
import os

In [7]:
from train_agent_classification_AL import Active_Learning_train


In [ ]:
for num_run in range(0,10):

    prev_num_run = num_run-1
    if num_run==0:
        resume_model_path = False
    else:
        resume_model_path = os.path.join(config['PROJECT']['group_dir'],'Stage_'+str(prev_num_run),'checkpoints','checkpoint.200.hdf5')
        
    stage_files = list(all_files[labeled_set])
    stage_labels = list(all_labels[labeled_set])
        
    Active_Learning = Active_Learning_train.remote(config, 
                                            stage_files,
                                            stage_labels,
                                            classes_semantics,
                                            num_run=num_run,
                                            resume_model_path=resume_model_path,
                                            resume = False)
    
    Active_Learning.start_training.remote()
    # Wait util the model is training
    
    while True:
        time.sleep(10)
        try:
            progress_id = Active_Learning.isTraining.remote()
            response = ray.get(progress_id)
            
            break
        except:
            pass

    # wait until the model finish training
    while True:
        time.sleep(10)
        progress_id = Active_Learning.isTraining.remote()
        response = ray.get(progress_id)
        
        get_progress = Active_Learning.get_progress.remote()
        print(ray.get(get_progress))
        
        if not response:
            break
    
    Active_Learning.__ray_terminate__.remote()
    
    del Active_Learning

    labeled_set += unlabeled_set[: add_images_stage]
    unlabeled_set = unlabeled_set[add_images_stage :]
    
    time.sleep(120)


(pid=10764, ip=192.168.8.52) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=10764, ip=192.168.8.52)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=10764, ip=192.168.8.52) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=10764, ip=192.168.8.52)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=10764, ip=192.168.8.52) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, i

(pid=10764, ip=192.168.8.52) Subset set to : train


(pid=10764, ip=192.168.8.52) 


(pid=10764, ip=192.168.8.52) ['Partially Dressed', 'Dressed', 'Explicit Dressed']
(pid=10764, ip=192.168.8.52) Train_Stage_0 The backbone is:  ResNet50


(pid=10764, ip=192.168.8.52) WARNING:tensorflow:From /mnt/Ressources/Andres/Temp_active/data_pipeline/Classification_dataset_AL.py:131: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
(pid=10764, ip=192.168.8.52) Instructions for updating:
(pid=10764, ip=192.168.8.52) tf.py_func is deprecated in TF V2. Instead, use
(pid=10764, ip=192.168.8.52)     tf.py_function, which takes a python function which manipulates tf eager
(pid=10764, ip=192.168.8.52)     tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
(pid=10764, ip=192.168.8.52)     an ndarray (just call tensor.numpy()) but having access to eager tensors
(pid=10764, ip=192.168.8.52)     means `tf.py_function`s can use accelerators such as GPUs as well as
(pid=10764, ip=192.168.8.52)     being differentiable using a gradient tape.
(pid=10764, ip=192.168.8.52)     
(pid=10764, ip=192.168.8.52) WARNING:tensorflow:From /home/reminiz/ReminizML2/python_env/lib/py

...

(pid=10956) Train_Stage_0 Init done
(pid=10956) Train_Stage_0 Start training


(pid=10956) WARNING:tensorflow:From /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
(pid=10956) Instructions for updating:
(pid=10956) Use tf.cast instead.


{'Current Epoch': 1, 'Total Epoch': 200, 'Current Step': 389, 'Total Steps': 31400, 'Progress': 1.23, 'steps_per_epoch': 157, 'status': 'Training'}
{'Current Epoch': 1, 'Total Epoch': 200, 'Current Step': 434, 'Total Steps': 31400, 'Progress': 1.38, 'steps_per_epoch': 157, 'status': 'Training'}
(pid=10956) Epoch 1/200
(pid=10764, ip=192.168.8.52)  - 35s - loss: 0.7895 - c_pred_loss: 0.7895 - l_pred_w_loss: 0.5408 - l_pred_s_loss: 0.5408 - c_pred_sparse_categorical_accuracy: 0.7592
(pid=10764, ip=192.168.8.52) Epoch 4/200
{'Current Epoch': 2, 'Total Epoch': 200, 'Current Step': 478, 'Total Steps': 31400, 'Progress': 1.52, 'steps_per_epoch': 157, 'status': 'Training'}


(pid=10956) 2021-01-12 17:50:51.196002: I tensorflow/stream_executor/dso_loader.cc:152] successfully opened CUDA library libcublas.so.10.0 locally


{'Current Epoch': 2, 'Total Epoch': 200, 'Current Step': 522, 'Total Steps': 31400, 'Progress': 1.66, 'steps_per_epoch': 157, 'status': 'Training'}
{'Current Epoch': 2, 'Total Epoch': 200, 'Current Step': 566, 'Total Steps': 31400, 'Progress': 1.8, 'steps_per_epoch': 157, 'status': 'Training'}
{'Current Epoch': 2, 'Total Epoch': 200, 'Current Step': 609, 'Total Steps': 31400, 'Progress': 1.94, 'steps_per_epoch': 157, 'status': 'Training'}
(pid=10764, ip=192.168.8.52)  - 36s - loss: 0.7688 - c_pred_loss: 0.7688 - l_pred_w_loss: 0.4971 - l_pred_s_loss: 0.4971 - c_pred_sparse_categorical_accuracy: 0.7584
(pid=10764, ip=192.168.8.52) Epoch 5/200
{'Current Epoch': 3, 'Total Epoch': 200, 'Current Step': 653, 'Total Steps': 31400, 'Progress': 2.07, 'steps_per_epoch': 157, 'status': 'Training'}
(pid=10956)  - 46s - loss: 1.2346 - c_pred_loss: 0.8469 - l_pred_w_loss: 0.3877 - l_pred_s_loss: 0.3877 - c_pred_sparse_categorical_accuracy: 0.7542
(pid=10956) Epoch 2/200
{'Current Epoch': 3, 'Total E